In [1]:
import ee
import geopandas as gpd
import geemap
import geersd
import pandas as pd

from pph import opca

In [2]:
ee.Initialize()

In [3]:
# load MM images
gdf = gpd.read_file("../../scratch/moose_mnt_l8_34_25_100.geojson")
gdf.shape

(82, 13)

In [4]:
gdf_roi = gpd.read_file("../../scratch/moose_mountain_FT_OPC_analysis_AOI_7year_L8.shp")
gdf_roi.set_crs(4326, inplace=True)
aoi = ee.FeatureCollection(gdf_roi.__geo_interface__).geometry()

In [5]:
gdf_2017 = gdf[(gdf.Year <= 2024) & (gdf.CLOUD_COVER < 6)]
gdf_2017

,date,system:id,system:time_end,system:time_start,CLOUD_COVER,UTM_ZONE,WRS_PATH,WRS_ROW,WRS_TYPE,Year,Month,Day,geometry
0,2017-04-05,LANDSAT/LC08/C02/T1_L2/LC08_034025_20170405,1491414014140,1491414014140,1.90,13,34,25,2,2017,4,5,"POLYGON ((-101.28906 49.20355, -101.27809 49.2..."
3,2017-05-23,LANDSAT/LC08/C02/T1_L2/LC08_034025_20170523,1495561213093,1495561213093,0.28,13,34,25,2,2017,5,23,"POLYGON ((-100.66959 50.47959, -100.64892 50.5..."
4,2017-06-08,LANDSAT/LC08/C02/T1_L2/LC08_034025_20170608,1496943621332,1496943621332,0.39,13,34,25,2,2017,6,8,"POLYGON ((-100.48309 50.89704, -100.48459 50.8..."
6,2017-07-10,LANDSAT/LC08/C02/T1_L2/LC08_034025_20170710,1499708429765,1499708429765,5.98,13,34,25,2,2017,7,10,"POLYGON ((-101.23794 49.20264, -101.23709 49.2..."
9,2017-08-27,LANDSAT/LC08/C02/T1_L2/LC08_034025_20170827,1503855647263,1503855647263,0.75,13,34,25,2,2017,8,27,"POLYGON ((-100.48388 50.89747, -100.51363 50.9..."
11,2017-09-28,LANDSAT/LC08/C02/T1_L2/LC08_034025_20170928,1506620454757,1506620454757,0.46,13,34,25,2,2017,9,28,"POLYGON ((-103.75152 49.64424, -103.75066 49.6..."
12,2017-10-14,LANDSAT/LC08/C02/T1_L2/LC08_034025_20171014,1508002859011,1508002859011,0.01,13,34,25,2,2017,10,14,"POLYGON ((-100.51155 50.89820, -100.51780 50.8..."
15,2018-04-24,LANDSAT/LC08/C02/T1_L2/LC08_034025_20180424,1524591595142,1524591595142,1.11,13,34,25,2,2018,4,24,"POLYGON ((-103.09943 51.34472, -103.10015 51.3..."
19,2018-06-27,LANDSAT/LC08/C02/T1_L2/LC08_034025_20180627,1530121182954,1530121182954,0.19,13,34,25,2,2018,6,27,"POLYGON ((-100.46559 50.89690, -100.46711 50.8..."
25,2018-10-17,LANDSAT/LC08/C02/T1_L2/LC08_034025_20181017,1539798032749,1539798032749,1.89,13,34,25,2,2018,10,17,"POLYGON ((-103.74851 49.64441, -103.74776 49.6..."


In [41]:
stack = None
for image in gdf_2017['system:id'].tolist():
    img = ee.Image(image).select('SR_B5')
    if stack is None:
        stack = img
    else:
        stack = stack.addBands(img)
    

In [42]:
stack

In [43]:
pca = opca.PrincipalCompoent().compute(stack, aoi)

In [44]:
pca.bandNames()

In [45]:
Map = geemap.Map()

Map.addLayer(pca.clip(aoi), {'min': -2, 'max': 2})
Map.centerObject(aoi, 10)
Map

Map(center=[49.76291503423149, -102.41828841732854], controls=(WidgetControl(options=['position', 'transparent…